In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.datasets import make_classification

X,y = make_classification(n_samples=100000, n_features=25, n_informative=20, n_redundant=5, random_state=0)

In [3]:
df = pd.DataFrame(X, columns = ["Column{}".format(i) for i in range(25)])

In [4]:
df["label"] = y

In [5]:
df.head()

,Column0,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,...,Column16,Column17,Column18,Column19,Column20,Column21,Column22,Column23,Column24,label
0,0.374795,2.409882,-3.785127,0.680042,1.762758,0.850472,-2.392243,-0.835476,1.082143,-1.253907,...,0.246301,0.252601,4.012128,0.088547,-1.327247,-2.174767,0.061074,-4.597709,-1.769054,1
1,5.294405,3.381126,0.628389,1.271747,-2.066395,-1.487487,-0.046180,-3.990081,0.824087,-3.305013,...,1.580230,-1.246286,-0.304895,1.726794,-0.056653,0.482031,-1.618998,-5.363979,-1.027658,1
2,-3.582907,0.657844,3.672484,0.723356,-4.947266,-0.471917,-0.352738,-11.224308,4.810373,10.842853,...,1.089028,-1.701240,2.196797,3.795722,-1.438148,-4.035236,-3.182302,0.212775,3.008187,0
3,-4.837864,0.080024,3.082630,-4.340017,1.924474,5.323912,-0.887652,-9.740558,0.278714,4.830757,...,-3.934576,-2.762419,-1.773916,-0.721765,-3.328776,-4.715541,2.241265,-4.451833,2.083070,1
4,6.869779,-1.221610,0.297304,-0.803456,-1.021820,-0.809890,0.041137,1.046422,-1.151566,-0.484239,...,1.485713,3.403892,0.261279,-3.216273,0.935161,0.678729,2.304216,8.114040,1.931248,0


In [6]:
features = df.drop("label", axis = 1)

In [7]:
features.shape

(100000, 25)

In [8]:
class_label = df["label"]

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(features, class_label, stratify=class_label ,random_state=42)

In [11]:
X_train.shape

(75000, 25)

In [12]:
y_train.shape

(75000,)

In [13]:
import time
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.model_selection import cross_val_score

In [14]:
# Import models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgb

In [15]:
accuracy = {}
roc_auc = {}
f1 = {}
cross_val_acc={}
diff_cv_acc = {}
speed_fit = {}
speed_cv = {}

### Decision Tree

In [16]:
dt_model = DecisionTreeClassifier()

start = time.time()

dt_model.fit(X_train,y_train)

pred_dt = dt_model.predict(X_test)

end = time.time()

speed_fit["fit_speed_dt"] = np.round( end - start, 3)

In [17]:
start = time.time()
score_dt = cross_val_score(estimator =dt_model,X = X_train.values, y = y_train, scoring="accuracy", cv=5, n_jobs=-1)
end = time.time()

speed_cv["cv_speed_dt"] = np.round( end - start, 3)
# Predict the test set probabilities of positive class
y_pred_proba_dt = dt_model.predict_proba(X_test)[:,1]

In [18]:
accuracy['Accuracy Score_dt'] = np.round(accuracy_score(y_test, pred_dt)*100, 3)
cross_val_acc["cross_val_acc_dt"] = np.round(np.mean(score_dt).round(3) * 100,3)
roc_auc['roc_auc_score_dt'] = np.round(roc_auc_score(y_test, y_pred_proba_dt) *100, 3)
f1['f1_score_dt'] = np.round(f1_score(y_test, pred_dt) * 100, 3)
diff_cv_acc["diff_cv_acc_dt"] = np.abs(np.round(accuracy['Accuracy Score_dt']-cross_val_acc["cross_val_acc_dt"],3))
print(accuracy, cross_val_acc, roc_auc, f1, diff_cv_acc, speed_fit, speed_cv)

{'Accuracy Score_dt': 86.432} {'cross_val_acc_dt': 86.3} {'roc_auc_score_dt': 86.432} {'f1_score_dt': 86.461} {'diff_cv_acc_dt': 0.132} {'fit_speed_dt': 3.171} {'cv_speed_dt': 5.295}


### Logistic Regression

In [19]:
lr_model = LogisticRegression(max_iter=1000)

start = time.time()

lr_model.fit(X_train,y_train)

pred_lr = lr_model.predict(X_test)

end = time.time()

speed_fit["fit_speed_lr"] = np.round( end - start, 3)

In [20]:
start = time.time()
score_lr = cross_val_score(estimator =lr_model,X =  X_train.values, y = y_train, scoring="accuracy", cv=5, n_jobs=-1)
end = time.time()
speed_cv["cv_speed_lr"] = np.round( end - start, 3)

# Predict the test set probabilities of positive class
y_pred_proba_lr = lr_model.predict_proba(X_test)[:,1]

In [22]:
accuracy['Accuracy Score_lr'] = np.round(accuracy_score(y_test, pred_lr)*100, 3)

cross_val_acc["cross_val_acc_lr"] = np.round(np.mean(score_lr).round(3) * 100,3)

roc_auc['roc_auc_score_lr'] = np.round(roc_auc_score(y_test, y_pred_proba_lr) *100, 3)

f1['f1_score_lr'] = np.round(f1_score(y_test, pred_lr) * 100, 3)

diff_cv_acc["diff_cv_acc_lr"] = np.round(np.abs(accuracy['Accuracy Score_lr']-cross_val_acc["cross_val_acc_lr"]), 3)

print(accuracy, cross_val_acc, roc_auc, f1, diff_cv_acc, speed_fit, speed_cv)

{'Accuracy Score_dt': 86.432, 'Accuracy Score_lr': 79.024} {'cross_val_acc_dt': 86.3, 'cross_val_acc_lr': 79.2} {'roc_auc_score_dt': 86.432, 'roc_auc_score_lr': 86.819} {'f1_score_dt': 86.461, 'f1_score_lr': 78.93} {'diff_cv_acc_dt': 0.132, 'diff_cv_acc_lr': 0.176} {'fit_speed_dt': 3.171, 'fit_speed_lr': 0.262} {'cv_speed_dt': 5.295, 'cv_speed_lr': 2.444}


### KNN

In [23]:
knn_model = KNeighborsClassifier()

start = time.time()

knn_model.fit(X_train,y_train)

pred_knn = knn_model.predict(X_test)

end = time.time()

speed_fit["fit_speed_knn"] = np.round( end - start, 3)

In [24]:
start = time.time()
score_knn = cross_val_score(estimator =knn_model,X =  X_train.values, y = y_train, scoring="accuracy", cv=5, n_jobs=-1)
end = time.time()
speed_cv["cv_speed_knn"] = np.round( end - start, 3)
# Predict the test set probabilities of positive class
y_pred_proba_knn = knn_model.predict_proba(X_test)[:,1]

In [25]:
accuracy['Accuracy Score_knn'] = np.round(accuracy_score(y_test, pred_knn)*100, 3)

cross_val_acc["cross_val_acc_knn"] = np.round(np.mean(score_knn).round(3) * 100,3)

roc_auc['roc_auc_score_knn'] = np.round(roc_auc_score(y_test, y_pred_proba_knn) *100, 3)

f1['f1_score_knn'] = np.round(f1_score(y_test, pred_knn) * 100, 3)

diff_cv_acc["diff_cv_acc_knn"] = np.round(np.abs(accuracy['Accuracy Score_knn']-cross_val_acc["cross_val_acc_knn"]), 3)

print(accuracy, cross_val_acc, roc_auc, f1, diff_cv_acc, speed_fit, speed_cv)

{'Accuracy Score_dt': 86.432, 'Accuracy Score_lr': 79.024, 'Accuracy Score_knn': 97.204} {'cross_val_acc_dt': 86.3, 'cross_val_acc_lr': 79.2, 'cross_val_acc_knn': 97.0} {'roc_auc_score_dt': 86.432, 'roc_auc_score_lr': 86.819, 'roc_auc_score_knn': 99.028} {'f1_score_dt': 86.461, 'f1_score_lr': 78.93, 'f1_score_knn': 97.201} {'diff_cv_acc_dt': 0.132, 'diff_cv_acc_lr': 0.176, 'diff_cv_acc_knn': 0.204} {'fit_speed_dt': 3.171, 'fit_speed_lr': 0.262, 'fit_speed_knn': 37.918} {'cv_speed_dt': 5.295, 'cv_speed_lr': 2.444, 'cv_speed_knn': 59.557}


### Random Forest

In [26]:
rf_model = RandomForestClassifier()

start = time.time()

rf_model.fit(X_train,y_train)

pred_rf = rf_model.predict(X_test)

end = time.time()

speed_fit["fit_speed_rf"] = np.round( end - start, 3)

In [27]:
start = time.time()
score_rf = cross_val_score(estimator =rf_model,X =  X_train.values, y = y_train, scoring="accuracy", cv=5, n_jobs=-1)
end = time.time()

speed_cv["cv_speed_rf"] = np.round( end - start, 3)

# Predict the test set probabilities of positive class
y_pred_proba_rf = rf_model.predict_proba(X_test)[:,1]

In [28]:
accuracy['Accuracy Score_rf'] = np.round(accuracy_score(y_test, pred_rf)*100, 3)

cross_val_acc["cross_val_acc_rf"] = np.round(np.mean(score_rf).round(3) * 100,3)

roc_auc['roc_auc_score_rf'] = np.round(roc_auc_score(y_test, y_pred_proba_rf) *100, 3)

f1['f1_score_rf'] = np.round(f1_score(y_test, pred_rf) * 100, 3)

diff_cv_acc["diff_cv_acc_rf"] = np.round(np.abs(accuracy['Accuracy Score_rf']-cross_val_acc["cross_val_acc_rf"]), 3)

print(accuracy, cross_val_acc, roc_auc, f1, diff_cv_acc, speed_fit, speed_cv)

{'Accuracy Score_dt': 86.432, 'Accuracy Score_lr': 79.024, 'Accuracy Score_knn': 97.204, 'Accuracy Score_rf': 96.608} {'cross_val_acc_dt': 86.3, 'cross_val_acc_lr': 79.2, 'cross_val_acc_knn': 97.0, 'cross_val_acc_rf': 96.3} {'roc_auc_score_dt': 86.432, 'roc_auc_score_lr': 86.819, 'roc_auc_score_knn': 99.028, 'roc_auc_score_rf': 99.058} {'f1_score_dt': 86.461, 'f1_score_lr': 78.93, 'f1_score_knn': 97.201, 'f1_score_rf': 96.588} {'diff_cv_acc_dt': 0.132, 'diff_cv_acc_lr': 0.176, 'diff_cv_acc_knn': 0.204, 'diff_cv_acc_rf': 0.308} {'fit_speed_dt': 3.171, 'fit_speed_lr': 0.262, 'fit_speed_knn': 37.918, 'fit_speed_rf': 37.452} {'cv_speed_dt': 5.295, 'cv_speed_lr': 2.444, 'cv_speed_knn': 59.557, 'cv_speed_rf': 30.596}


### AdaBoost

In [29]:
ada_model = AdaBoostClassifier(n_estimators=100)

start = time.time()

ada_model.fit(X_train,y_train)

pred_ada = ada_model.predict(X_test)

end = time.time()

speed_fit["fit_speed_ada"] = np.round( end - start, 3)

In [30]:
start = time.time()
score_ada = cross_val_score(estimator =ada_model,X =  X_train.values, y = y_train, scoring="accuracy", cv=5, n_jobs=-1)
end = time.time()

speed_cv["cv_speed_ada"] = np.round( end - start, 3)

# Predict the test set probabilities of positive class
y_pred_proba_ada = ada_model.predict_proba(X_test)[:,1]

In [31]:
accuracy['Accuracy Score_ada'] = np.round(accuracy_score(y_test, pred_ada)*100, 3)

cross_val_acc["cross_val_acc_ada"] = np.round(np.mean(score_ada).round(3) * 100,3)

roc_auc['roc_auc_score_ada'] = np.round(roc_auc_score(y_test, y_pred_proba_ada) *100, 3)

f1['f1_score_ada'] = np.round(f1_score(y_test, pred_ada) * 100, 3)

diff_cv_acc["diff_cv_acc_ada"] = np.round(np.abs(accuracy['Accuracy Score_ada']-cross_val_acc["cross_val_acc_ada"]), 3)

print(accuracy, cross_val_acc, roc_auc, f1, diff_cv_acc, speed_fit, speed_cv)

{'Accuracy Score_dt': 86.432, 'Accuracy Score_lr': 79.024, 'Accuracy Score_knn': 97.204, 'Accuracy Score_rf': 96.608, 'Accuracy Score_ada': 81.14} {'cross_val_acc_dt': 86.3, 'cross_val_acc_lr': 79.2, 'cross_val_acc_knn': 97.0, 'cross_val_acc_rf': 96.3, 'cross_val_acc_ada': 81.4} {'roc_auc_score_dt': 86.432, 'roc_auc_score_lr': 86.819, 'roc_auc_score_knn': 99.028, 'roc_auc_score_rf': 99.058, 'roc_auc_score_ada': 89.467} {'f1_score_dt': 86.461, 'f1_score_lr': 78.93, 'f1_score_knn': 97.201, 'f1_score_rf': 96.588, 'f1_score_ada': 81.154} {'diff_cv_acc_dt': 0.132, 'diff_cv_acc_lr': 0.176, 'diff_cv_acc_knn': 0.204, 'diff_cv_acc_rf': 0.308, 'diff_cv_acc_ada': 0.26} {'fit_speed_dt': 3.171, 'fit_speed_lr': 0.262, 'fit_speed_knn': 37.918, 'fit_speed_rf': 37.452, 'fit_speed_ada': 28.579} {'cv_speed_dt': 5.295, 'cv_speed_lr': 2.444, 'cv_speed_knn': 59.557, 'cv_speed_rf': 30.596, 'cv_speed_ada': 27.657}


### Gradient Boosting

In [32]:
gb_model = GradientBoostingClassifier()

start = time.time()

gb_model.fit(X_train,y_train)

pred_gb = gb_model.predict(X_test)

end = time.time()

speed_fit["fit_speed_gb"] = np.round( end - start, 3)

In [33]:
start = time.time()
score_gb = cross_val_score(estimator =gb_model,X =  X_train.values, y = y_train, scoring="accuracy", cv=5, n_jobs=-1)
end = time.time()

speed_cv["cv_speed_gb"] = np.round( end - start, 3)

# Predict the test set probabilities of positive class
y_pred_proba_gb = gb_model.predict_proba(X_test)[:,1]

In [34]:
accuracy['Accuracy Score_gb'] = np.round(accuracy_score(y_test, pred_gb)*100, 3)

cross_val_acc["cross_val_acc_gb"] = np.round(np.mean(score_gb).round(3) * 100,3)

roc_auc['roc_auc_score_gb'] = np.round(roc_auc_score(y_test, y_pred_proba_gb) *100, 3)

f1['f1_score_gb'] = np.round(f1_score(y_test, pred_gb) * 100, 3)

diff_cv_acc["diff_cv_acc_gb"] = np.round(np.abs(accuracy['Accuracy Score_gb']-cross_val_acc["cross_val_acc_gb"]), 3)

print(accuracy, cross_val_acc, roc_auc, f1, diff_cv_acc, speed_fit, speed_cv)

{'Accuracy Score_dt': 86.432, 'Accuracy Score_lr': 79.024, 'Accuracy Score_knn': 97.204, 'Accuracy Score_rf': 96.608, 'Accuracy Score_ada': 81.14, 'Accuracy Score_gb': 88.928} {'cross_val_acc_dt': 86.3, 'cross_val_acc_lr': 79.2, 'cross_val_acc_knn': 97.0, 'cross_val_acc_rf': 96.3, 'cross_val_acc_ada': 81.4, 'cross_val_acc_gb': 89.2} {'roc_auc_score_dt': 86.432, 'roc_auc_score_lr': 86.819, 'roc_auc_score_knn': 99.028, 'roc_auc_score_rf': 99.058, 'roc_auc_score_ada': 89.467, 'roc_auc_score_gb': 95.763} {'f1_score_dt': 86.461, 'f1_score_lr': 78.93, 'f1_score_knn': 97.201, 'f1_score_rf': 96.588, 'f1_score_ada': 81.154, 'f1_score_gb': 88.821} {'diff_cv_acc_dt': 0.132, 'diff_cv_acc_lr': 0.176, 'diff_cv_acc_knn': 0.204, 'diff_cv_acc_rf': 0.308, 'diff_cv_acc_ada': 0.26, 'diff_cv_acc_gb': 0.272} {'fit_speed_dt': 3.171, 'fit_speed_lr': 0.262, 'fit_speed_knn': 37.918, 'fit_speed_rf': 37.452, 'fit_speed_ada': 28.579, 'fit_speed_gb': 68.417} {'cv_speed_dt': 5.295, 'cv_speed_lr': 2.444, 'cv_speed_kn

### Light GBM

In [35]:
lgb_model = lgb.LGBMClassifier()

start = time.time()

lgb_model.fit(X_train,y_train)

pred_lgb = lgb_model.predict(X_test)

end = time.time()

speed_fit["fit_speed_lgb"] = np.round(end - start, 3)

In [36]:
start = time.time()
score_lgb = cross_val_score(estimator = lgb_model, X =  X_train.values, y = y_train, scoring="accuracy", cv=5, n_jobs=-1)
end = time.time()

speed_cv["cv_speed_lgb"] = np.round( end - start, 3)

# Predict the test set probabilities of positive class
y_pred_proba_lgb = lgb_model.predict_proba(X_test)[:,1]

In [37]:
accuracy['Accuracy Score_lgb'] = np.round(accuracy_score(y_test, pred_lgb)*100, 3)

cross_val_acc["cross_val_acc_lgb"] = np.round(np.mean(score_lgb).round(3) * 100,3)

roc_auc['roc_auc_score_lgb'] = np.round(roc_auc_score(y_test, y_pred_proba_lgb) *100, 3)

f1['f1_score_lgb'] = np.round(f1_score(y_test, pred_lgb) * 100, 3)

diff_cv_acc["diff_cv_acc_lgb"] = np.round(np.abs(accuracy['Accuracy Score_lgb']-cross_val_acc["cross_val_acc_lgb"]), 3)

print(accuracy, cross_val_acc, roc_auc, f1, diff_cv_acc, speed_fit, speed_cv)

{'Accuracy Score_dt': 86.432, 'Accuracy Score_lr': 79.024, 'Accuracy Score_knn': 97.204, 'Accuracy Score_rf': 96.608, 'Accuracy Score_ada': 81.14, 'Accuracy Score_gb': 88.928, 'Accuracy Score_lgb': 95.508} {'cross_val_acc_dt': 86.3, 'cross_val_acc_lr': 79.2, 'cross_val_acc_knn': 97.0, 'cross_val_acc_rf': 96.3, 'cross_val_acc_ada': 81.4, 'cross_val_acc_gb': 89.2, 'cross_val_acc_lgb': 95.8} {'roc_auc_score_dt': 86.432, 'roc_auc_score_lr': 86.819, 'roc_auc_score_knn': 99.028, 'roc_auc_score_rf': 99.058, 'roc_auc_score_ada': 89.467, 'roc_auc_score_gb': 95.763, 'roc_auc_score_lgb': 98.858} {'f1_score_dt': 86.461, 'f1_score_lr': 78.93, 'f1_score_knn': 97.201, 'f1_score_rf': 96.588, 'f1_score_ada': 81.154, 'f1_score_gb': 88.821, 'f1_score_lgb': 95.498} {'diff_cv_acc_dt': 0.132, 'diff_cv_acc_lr': 0.176, 'diff_cv_acc_knn': 0.204, 'diff_cv_acc_rf': 0.308, 'diff_cv_acc_ada': 0.26, 'diff_cv_acc_gb': 0.272, 'diff_cv_acc_lgb': 0.292} {'fit_speed_dt': 3.171, 'fit_speed_lr': 0.262, 'fit_speed_knn': 37

### XGBoost

In [38]:
xgb_model = xgb.XGBClassifier(objective='binary:logistic')

start = time.time()

xgb_model.fit(X_train,y_train)

pred_xgb = xgb_model.predict(X_test)

end = time.time()

speed_fit["fit_speed_xgb"] = np.round(end - start, 3)

[17:16:57] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\User\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [39]:
start = time.time()
score_xgb = cross_val_score(estimator = xgb_model, X =  X_train.values, y = y_train, scoring="accuracy", cv=5, n_jobs=-1)
end = time.time()

speed_cv["cv_speed_xgb"] = np.round( end - start, 3)

# Predict the test set probabilities of positive class
y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:,1]

In [40]:
accuracy['Accuracy Score_xgb'] = np.round(accuracy_score(y_test, pred_xgb)*100, 3)

cross_val_acc["cross_val_acc_xgb"] = np.round(np.mean(score_xgb).round(3) * 100,3)

roc_auc['roc_auc_score_xgb'] = np.round(roc_auc_score(y_test, y_pred_proba_xgb) *100, 3)

f1['f1_score_xgb'] = np.round(f1_score(y_test, pred_xgb) * 100, 3)

diff_cv_acc["diff_cv_acc_xgb"] = np.round(np.abs(accuracy['Accuracy Score_xgb']-cross_val_acc["cross_val_acc_xgb"]), 3)

print(accuracy, cross_val_acc, roc_auc, f1, diff_cv_acc, speed_fit, speed_cv)

{'Accuracy Score_dt': 86.432, 'Accuracy Score_lr': 79.024, 'Accuracy Score_knn': 97.204, 'Accuracy Score_rf': 96.608, 'Accuracy Score_ada': 81.14, 'Accuracy Score_gb': 88.928, 'Accuracy Score_lgb': 95.508, 'Accuracy Score_xgb': 97.144} {'cross_val_acc_dt': 86.3, 'cross_val_acc_lr': 79.2, 'cross_val_acc_knn': 97.0, 'cross_val_acc_rf': 96.3, 'cross_val_acc_ada': 81.4, 'cross_val_acc_gb': 89.2, 'cross_val_acc_lgb': 95.8, 'cross_val_acc_xgb': 97.1} {'roc_auc_score_dt': 86.432, 'roc_auc_score_lr': 86.819, 'roc_auc_score_knn': 99.028, 'roc_auc_score_rf': 99.058, 'roc_auc_score_ada': 89.467, 'roc_auc_score_gb': 95.763, 'roc_auc_score_lgb': 98.858, 'roc_auc_score_xgb': 99.241} {'f1_score_dt': 86.461, 'f1_score_lr': 78.93, 'f1_score_knn': 97.201, 'f1_score_rf': 96.588, 'f1_score_ada': 81.154, 'f1_score_gb': 88.821, 'f1_score_lgb': 95.498, 'f1_score_xgb': 97.139} {'diff_cv_acc_dt': 0.132, 'diff_cv_acc_lr': 0.176, 'diff_cv_acc_knn': 0.204, 'diff_cv_acc_rf': 0.308, 'diff_cv_acc_ada': 0.26, 'diff_c

In [41]:
accuracy

{'Accuracy Score_dt': 86.432,
 'Accuracy Score_lr': 79.024,
 'Accuracy Score_knn': 97.204,
 'Accuracy Score_rf': 96.608,
 'Accuracy Score_ada': 81.14,
 'Accuracy Score_gb': 88.928,
 'Accuracy Score_lgb': 95.508,
 'Accuracy Score_xgb': 97.144}

In [42]:
cross_val_acc

{'cross_val_acc_dt': 86.3,
 'cross_val_acc_lr': 79.2,
 'cross_val_acc_knn': 97.0,
 'cross_val_acc_rf': 96.3,
 'cross_val_acc_ada': 81.4,
 'cross_val_acc_gb': 89.2,
 'cross_val_acc_lgb': 95.8,
 'cross_val_acc_xgb': 97.1}

In [43]:
roc_auc

{'roc_auc_score_dt': 86.432,
 'roc_auc_score_lr': 86.819,
 'roc_auc_score_knn': 99.028,
 'roc_auc_score_rf': 99.058,
 'roc_auc_score_ada': 89.467,
 'roc_auc_score_gb': 95.763,
 'roc_auc_score_lgb': 98.858,
 'roc_auc_score_xgb': 99.241}

In [44]:
f1

{'f1_score_dt': 86.461,
 'f1_score_lr': 78.93,
 'f1_score_knn': 97.201,
 'f1_score_rf': 96.588,
 'f1_score_ada': 81.154,
 'f1_score_gb': 88.821,
 'f1_score_lgb': 95.498,
 'f1_score_xgb': 97.139}

In [45]:
diff_cv_acc

{'diff_cv_acc_dt': 0.132,
 'diff_cv_acc_lr': 0.176,
 'diff_cv_acc_knn': 0.204,
 'diff_cv_acc_rf': 0.308,
 'diff_cv_acc_ada': 0.26,
 'diff_cv_acc_gb': 0.272,
 'diff_cv_acc_lgb': 0.292,
 'diff_cv_acc_xgb': 0.044}

In [46]:
speed_fit

{'fit_speed_dt': 3.171,
 'fit_speed_lr': 0.262,
 'fit_speed_knn': 37.918,
 'fit_speed_rf': 37.452,
 'fit_speed_ada': 28.579,
 'fit_speed_gb': 68.417,
 'fit_speed_lgb': 0.472,
 'fit_speed_xgb': 7.757}

In [47]:
speed_cv

{'cv_speed_dt': 5.295,
 'cv_speed_lr': 2.444,
 'cv_speed_knn': 59.557,
 'cv_speed_rf': 30.596,
 'cv_speed_ada': 27.657,
 'cv_speed_gb': 60.938,
 'cv_speed_lgb': 2.801,
 'cv_speed_xgb': 30.596}